# importing libraries

In [ ]:
#!python -m ipykernel install --user --name="Sales Forecasting"

In [ ]:
from platform import python_version

print(python_version())


In [ ]:
import numpy as np
import tensorflow
np.__version__, tensorflow.__version__

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

#import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
# from keras.optimizers import Adam .
from tensorflow.keras.optimizers import Adam # - Works
from tensorflow.keras.callbacks import EarlyStopping
#from keras.utils import np_utils
from tensorflow.keras.layers import LSTM
from sklearn.model_selection import KFold, cross_val_score, train_test_split


In [ ]:
# !pip uninstall xlrd

#!pip install openpyxl
# !pip install xlrd==1.2.0

In [ ]:
df = pd.read_excel('Data Set.xlsx', engine='openpyxl') #reading data excel file 

In [ ]:
df

In [ ]:
df.info()

# Preprocessing Data

In [ ]:
#df = df.loc[17:,:]

In [ ]:
#df.reset_index(inplace = True )

In [ ]:
#df.drop('index',axis = 1, inplace = True)

In [ ]:
# df.columns = df.loc[0]
# df.columns

In [ ]:
# df.drop(df.index[0], inplace = True)

In [ ]:
# df.reset_index(inplace=True)

In [ ]:
# df.drop('index',axis = 1, inplace=True)

In [ ]:
df.tail()

In [ ]:
df.ffill(inplace=True)
df.head(110)

In [ ]:
df=df[~df['Date'].str.contains("Total")]
df.head(110)

In [ ]:
df=df[~df['Date'].str.contains("Total")]
df.head(107)

In [ ]:
df=df[['Date','Item Name',"Qty Total (Cup)"]]
df.head(40)

In [ ]:
# df.head(50)

In [ ]:
df["Date"] = pd.to_datetime(df["Date"])
df["Date"]=df['Date'].dt.strftime('%d/%m/%Y')
sum_df = df.groupby(['Date','Item Name']).agg({'Qty Total (Cup)': 'sum'})
sum_df=sum_df.reset_index()
sum_df["Date"] = pd.to_datetime(sum_df["Date"],infer_datetime_format=True)
sum_df.sort_values(by=['Date'], inplace=True, ascending=True)
sum_df["Date"]=sum_df['Date'].dt.strftime('%Y-%m-%d')
sum_df.head(10)

In [ ]:
sum_df = sum_df.reset_index(drop=True)
sum_df

In [ ]:
final_df = sum_df.copy()

In [ ]:
final_df = final_df.groupby(['Item Name']).agg({'Qty Total (Cup)': 'sum'})

In [ ]:
final_df

# Sales record of Brown Sugar Milk Tea

In [ ]:
data = sum_df[sum_df['Item Name'].str.contains("Brown Sugar Milk Tea")]
data

In [ ]:
data.rename(columns = {'Date':'date','Qty Total (Cup)':'sales'}, inplace = True)
df_sales = data.copy()
df_sales = df_sales[['date','sales']]

In [ ]:
df_sales.reset_index(inplace=True, drop=True)
df_sales

In [ ]:
df_sales.info()

In [ ]:
df_sales['date'] = pd.to_datetime(df_sales['date']) 
df_sales.info()

In [ ]:
df_sales = df_sales.iloc[17:-21]

In [ ]:
df_sales.reset_index(inplace=True, drop=True)
df_sales

In [ ]:
df_sales = df_sales.drop_duplicates('date')

In [ ]:
df_sales['sales'].plot()
plt.show()

In [ ]:
df_sales['sales'].hist()
plt.show()

In [ ]:
sns.boxplot(df_sales['sales'])
plt.show()

In [ ]:
## handling outliers
outliers = []
def detect_outliers_iqr(data):
    data = sorted(data)
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    # print(q1, q3)
    IQR = q3-q1
    lwr_bound = q1-(1.5*IQR)
    upr_bound = q3+(1.5*IQR)
    # print(lwr_bound, upr_bound)
    for i in data: 
        if (i<lwr_bound or i>upr_bound):
            outliers.append(i)
    return outliers# Driver code
sample_outliers = detect_outliers_iqr(df_sales['sales'])
print("Outliers from IQR method: ", sample_outliers)

median = np.median(df_sales['sales'])# Replace with median
sales = np.array(list(df_sales['sales']))
#print(len(sales))
#c = None
for i in sample_outliers:
    df_sales['sales'] = np.where(df_sales['sales']==i, int(median), df_sales['sales'])
    #print(i)
    #print(c)
    #print('-----')
print("Sample: ", median)
print("New array: ", df_sales['sales'])
#print(x.dtype)

In [ ]:
sns.boxplot(df_sales['sales'])
plt.show()

In [ ]:
df_sales['sales'].plot()
plt.show()

In [ ]:
### Testing For Stationarity
from statsmodels.tsa.stattools import adfuller

In [ ]:
#Ho: It is non stationary
#H1: It is stationary
def adfuller_test(sales):
    result=adfuller(sales)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

In [ ]:
adfuller_test(df_sales['sales'])

In [ ]:
#create a new dataframe to model the difference
# df_diff = df_sales.copy()

In [ ]:
# df_sales['Seasonal First Difference']=df_sales['sales'] - df_sales['sales'].shift(7)

In [ ]:
# ## Again test dickey fuller test
# adfuller_test(df_sales['Seasonal First Difference'].dropna())

In [ ]:
# df_sales['Seasonal First Difference'].plot()
# plt.show()

In [ ]:
df_sales.index = df_sales['date']
df_sales.columns

In [ ]:
df_sales.drop('date', axis=1, inplace=True)
df_sales

In [ ]:
df_sales = df_sales.fillna(method='bfill').fillna(method='ffill')

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
sales = df_sales.sales.asfreq('d')

In [ ]:
df_sales.isnull().sum()

In [ ]:
sales = sales.fillna(method='bfill').fillna(method='ffill')

In [ ]:
results = seasonal_decompose(sales)
results.plot()

In [ ]:
df_sales

In [ ]:
train = df_sales[:-30]
test = df_sales[-30:]

In [ ]:
train

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
scaler.fit(train)
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

In [ ]:
scaled_train[:10]

In [ ]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
# define generator
n_input = 7
n_features = 1
generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)

In [ ]:
X,y = generator[0]
print(f'Given the Array: \n{X.flatten()}')
print(f'Predict this y: \n {y}')

In [ ]:
X.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [ ]:
n_input

In [ ]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

In [ ]:
model.fit(generator,epochs=20)

In [ ]:
loss_per_epoch = model.history.history['loss']
plt.plot(range(len(loss_per_epoch)),loss_per_epoch)

In [ ]:
last_train_batch = scaled_train[-7:]

In [ ]:
last_train_batch = last_train_batch.reshape((1, n_input, n_features))

In [ ]:
model.predict(last_train_batch)

In [ ]:
scaled_test[0]

In [ ]:
df_sales[-7:].to_csv('examplers.csv')

In [ ]:
model.save('model.h5')

In [ ]:
import joblib
filename = 'scaler.sav'
joblib.dump(scaler, filename)

In [ ]:
model = tensorflow.keras.models.load_model('model.h5')

In [ ]:
test_df = pd.read_csv('examplers.csv')
test_df.set_index('date', inplace=True)
test_df

In [ ]:
#scaled_train = scaler.transform(train)
scaler_load = joblib.load('scaler.sav')
scaled_test = scaler_load.transform(test_df)

In [ ]:
scaled_test

In [ ]:
test_predictions = []

first_eval_batch = scaled_test
current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(len(test_df)):
    
    # get the prediction value for the first batch
    current_pred = model.predict(current_batch)[0]
    
    # append the prediction into the array
    test_predictions.append(current_pred) 
    
    # use the prediction to update the batch and remove the first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
test_predictions

In [ ]:
true_predictions = scaler_load.inverse_transform(test_predictions)

In [ ]:
true_predictions.flatten()

In [ ]:
import datetime
ind = test_df.tail(1).index
date_indices = pd.date_range(ind[0], periods=8)
exemplers_df = pd.DataFrame({'sales': list(np.round(true_predictions.flatten(), 2))}, index=date_indices[1:])
exemplers_df.index.name = 'date'
print(exemplers_df)
exemplers_df.to_csv('examplers.csv')

In [ ]:
test_df['Predictions'] = true_predictions

In [ ]:
df_ = pd.DataFrame(test_df['Predictions'])
#df_.to_csv

In [ ]:
test

In [ ]:
df_.plot(figsize=(14,5))

In [ ]:
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(df_sales['sales'])
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = plot_acf(df_sales['Seasonal First Difference'].iloc[200:],lags=40,ax=ax1)
ax2 = fig.add_subplot(212)
fig = plot_pacf(df_sales['Seasonal First Difference'].iloc[200:],lags=40,ax=ax2)
plt.show(fig)

In [ ]:
# For non-seasonal data
#p=1, d=1, q=0 or 1
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
model=ARIMA(df_sales['sales'],order=(1,1,0))
model_fit=model.fit()

In [ ]:
model_fit.summary()

In [ ]:
df_sales['forecast']=model_fit.predict(start=90,end=300,dynamic=True)
df_sales[['sales','forecast']].plot(figsize=(12,8))
plt.show()

In [ ]:
import statsmodels.api as sm
model=sm.tsa.statespace.SARIMAX(df_sales['sales'],order=(1, 1, 1),seasonal_order=(1,1,1,200))
results=model.fit()

In [ ]:
df_sales['forecast']=results.predict(start=201,end=270,dynamic=True)
df_sales[['sales','forecast']].plot(figsize=(12,8))
plt.show()

In [ ]:
#add previous sales to the next row
df_diff['prev_sales'] = df_diff['sales'].shift(1)
#drop the null values and calculate the difference
df_diff = df_diff.dropna()

df_diff['diff'] = (df_diff['sales'] - df_diff['prev_sales'])

#create new dataframe from transformation from time series to supervised
df_supervised = df_diff.drop(['prev_sales'],axis=1)
df_supervised= df_supervised.append(pd.DataFrame({'date': pd.date_range(start=df_supervised.date.iloc[-1], periods=6, freq='D', closed='right')}))
#adding lags
for inc in range(1,13):
    field_name = 'lag_' + str(inc)
    df_supervised[field_name] = df_supervised['diff'].shift(inc)
    
#drop null values
df_supervised = df_supervised.dropna().reset_index(drop=True)


In [ ]:
df_supervised

In [ ]:
extraDates=pd.DataFrame({'date': pd.date_range(start=df_supervised.date.iloc[-1], periods=6, freq='d', closed='right')})
fdf=df_supervised.append(extraDates)
fdf['date'] = pd.to_datetime(fdf['date'], errors='coerce')
fdf["date"]=fdf['date'].dt.strftime('%Y-%m-%d')
fdf=fdf.reset_index()
fdf = fdf.drop(['index'],axis=1)

fdf.tail(20)
fdf=fdf.fillna(0)

In [ ]:
fdf

In [ ]:
# Import statsmodels.formula.api
import statsmodels.formula.api as smf 

# Define the regression formula
model = smf.ols(formula='diff ~ lag_1', data=df_supervised)

# Fit the regression
model_fit = model.fit()

# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

In [ ]:

# Import statsmodels.formula.api
import statsmodels.formula.api as smf 

# Define the regression formula
model = smf.ols(formula='diff ~ lag_1 + lag_2 + lag_3 + lag_4 + lag_5', data=df_supervised)

# Fit the regression
model_fit = model.fit()

# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

In [ ]:
# Import statsmodels.formula.api
import statsmodels.formula.api as smf 

# Define the regression formula
model = smf.ols(formula='diff ~ lag_1 + lag_2 + lag_3 + lag_4 + lag_5 + lag_6 + lag_7 + lag_8 + lag_9 + lag_10 + lag_11 + lag_12', data=df_supervised)

# Fit the regression
model_fit = model.fit()

# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

In [ ]:
#import MinMaxScaler and create a new dataframe for LSTM model
from sklearn.preprocessing import MinMaxScaler
df_model = fdf.drop(['sales','date'],axis=1)

In [ ]:
df_model

In [ ]:
#split train and test set
train_set, test_set = df_model[0:-50].values, df_model[-50:].values

In [ ]:
#apply Min Max Scaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(train_set)
# reshape training set
train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])
train_set_scaled = scaler.transform(train_set)

# reshape test set
test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
test_set_scaled = scaler.transform(test_set)

In [ ]:
train_set_scaled

In [ ]:
X_train, y_train = train_set_scaled[:, 1:], train_set_scaled[:, 0:1]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])

In [ ]:
X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1]
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# Training LSTM  model

In [ ]:
model = Sequential()
model.add(LSTM(128, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae'])
model.fit(X_train, y_train, epochs=100, batch_size=1, validation_data=(X_test, y_test),verbose=1, shuffle=False)

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
from tensorflow.keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
model=loaded_model

In [ ]:
X_test

In [ ]:
y_pred = model.predict(X_test,batch_size=1)
y_pred

In [ ]:
#reshape y_pred
y_pred = y_pred.reshape(y_pred.shape[0], 1, y_pred.shape[1])

In [ ]:
import numpy as np

In [ ]:
#rebuild test set for inverse transform
pred_test_set = []
for index in range(0,len(y_pred)):
    print(np.concatenate([y_pred[index],X_test[index]],axis=1))
    pred_test_set.append(np.concatenate([y_pred[index],X_test[index]],axis=1))

In [ ]:
#reshape pred_test_set
pred_test_set = np.array(pred_test_set)
pred_test_set = pred_test_set.reshape(pred_test_set.shape[0], pred_test_set.shape[2])

In [ ]:
#inverse transform
pred_test_set_inverted = scaler.inverse_transform(pred_test_set)

In [ ]:
#create dataframe that shows the predicted sales
result_list = []
sales_dates = list(df_sales[-51:].date)
act_sales = list(df_sales[-51:].sales)
# pred_test_set_inverted[index][0] 

In [ ]:
sales_dates

In [ ]:
for index in range(0,len(pred_test_set_inverted)):
    result_dict = {}
    print(index)
    result_dict['pred_value'] = int(pred_test_set_inverted[index][0] + act_sales[index])
    result_dict['date'] = sales_dates[index+1]
    result_list.append(result_dict)
df_result = pd.DataFrame(result_list)

In [ ]:
#merge with actual sales dataframe
df_sales_pred = pd.merge(df_sales,df_result,on='date',how='left')

In [ ]:
df_sales_pred.tail(50)

In [ ]:
from matplotlib.pyplot import figure

figure(figsize=(20, 10), dpi=80)
plt.plot(df_sales_pred['date'], df_sales_pred['sales'], label = "actual")
plt.plot(df_sales_pred['date'], df_sales_pred['pred_value'], label = "predicted")
# plt.plot(x, y, label = "line 1")
# plt.plot(y, x, label = "line 2")
plt.xlabel("Dates")
plt.ylabel("Sales")
plt.legend()
plt.show()
